### Задача 3.
Даны файлы 1, 2, 123. Допустим это логи некоторых активностей (не играет значения что это), со временем активности и географическими координатами

**А)** Сделать датафрейм с попарным сравнением расстояния между всеми активностями в файлах 1, 2\
Вот статья про перевод разницы градусов долгот/широт в километры\
https://www.datafix.com.au/BASHing/2018-11-07.html \
Если будут сложности с реализацией, то будет дан прекод про расчету расстояния

**Б)** Для файла 123 подготовить функцию, которая будет по задаваемым параметрам координат и времени (дата и часы) генерировать строки со следующими аттрибутами\
1) Время в часах от предыдущей активности по времени активности до заданного времени (только число).
Например, задали дату, время - '2021-10-01 13:00', то в таблице ищется ближайшее по времени событие перед заданным временем;\
2) Расстояние в метрах от предыдущей активности до заданной координаты; \
3) Время после заданного и до ближайшей активности после заданного времени до ближайшей активности (по аналогии с ближайшим событием перед заданным временем из п. 1, 2).Если последующей активности нет, то ничего не проставлять, то есть если введено время, после которого нет залогированного события;\
4) Расстояние в метрах от заданных координат до координат следующей активности;\
5) Количество активностей в радиусе 1км от заданных координат в течение одного года до заданного времени;\
6) Количество активностей в радиусе 1км от заданных координат в течение одного года после заданного времени.

Подразумевается, что функция за один вызов генерирует одну строчку.

С такими признаками в цикле сгенерировать не менеее 30тыс. объектов (строк) по следующему принципу:\
В цикле рандомно генерируются координаты и время с некоторыми шумом, т.е. некоторая величина из какого-либо распределения с определенным средним и разбросом (дисперсией)\
(не важно какие параметры распределения, среднего и разброса (дисперсии), главное чтобы их было легко изменить в коде).

Примеры координат и времени разной удаленности

55.6734505, 37.464512
55.6745226, 37.463415
55.8604535, 37.702246
55.8724547, 37.864217
56.0734505, 38.064510
53.0734505, 35.064511
51.0734505, 37.064511

'2021-06-20 16:00'
'2021-06-20 10:00'
'2021-06-20 19:00'
'2021-06-20 23:00'
'2021-06-19 23:00'
'2021-06-21 01:00'
'2021-06-23 15:00'
'2021-06-27 11:00'
'2021-06-01 12:00'
'2021-09-01 13:00'

Соответственно в цикле вызывается вышеописанная функция с созданием признаков.

При этом для каждого объекта (строчки) в цикле необходимо проставить 1 в случае, если ближащие по времени события (события до и после - первые 4 признака) были близко по времени (допустим, в течение нескольких недель) и близко по расстоянию (допустим в пределах 15км) и если были активности в течение одного года до / после заданного времени в радиусе 1км (признаки 5, 6).\
Для каждого объекта (строчки) необходимо проставить 0, если ближащие события были близко по времени (допустим, в течение нескольких часов), а расстояние от ближайших по времени активностей большое, например, более 100км и за всю историю не было активностей. Для остальных признаков расстановка осуществляется рандомно.\
Здесь внимание, качество проставления единиц и нулей оценивается слабо, т.е. если ты дошел до этого этапа и не знаешь быстро как это сделать, то это не будет минусом.


In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import math
import random
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
table1 = pd.read_csv('~/!Tests/1.csv', sep=';', parse_dates=['Дата операции'])
table2 = pd.read_csv('~/!Tests/2.csv', sep=';', parse_dates=['Дата операции'])
table123 = pd.read_csv('~/!Tests/123.csv', sep=';', parse_dates=['Дата операции'])

In [3]:
datasets = [table1,table2,table123]
for df in datasets:
    print(df.info(), '\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Дата операции  1143 non-null   datetime64[ns]
 1   Координаты     1105 non-null   object        
dtypes: datetime64[ns](1), object(1)
memory usage: 18.0+ KB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Дата операции  189 non-null    datetime64[ns]
 1   Координаты     178 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 3.1+ KB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332 entries, 0 to 1331
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Дата операции  1332 non-null   datet

Во всех датасетах есть пропущенные значения по координатам, удалим их.

In [4]:
table1 = table1.dropna()
table2 = table2.dropna()
table123 = table123.dropna()

### Задание 3а

Создадим функцию для расчета расстояния.

In [5]:
def distance(lat1,long1,lat2,long2):
    pi = 3.14159
    x = 111.32 * (lat2 - lat1)
    y = 111.32 * (long2 - long1) * math.cos(lat1 * (pi / 180))  

    return math.sqrt(x**2 + y**2)

Получим итоговый отчет по расстояниям между активностями, вставляя пропуски, где координаты не определены.

In [6]:
result = table1.merge(table2, how='cross', suffixes=('_t1','_t2'))
result['distance'] = result.apply(lambda df: distance(
float([df['Координаты_t1'].split(' , ')[0] if df['Координаты_t1'] == df['Координаты_t1'] else np.nan][0]),
float([df['Координаты_t1'].split(' , ')[1] if df['Координаты_t1'] == df['Координаты_t1'] else np.nan][0]),
float([df['Координаты_t2'].split(' , ')[0] if df['Координаты_t2'] == df['Координаты_t2'] else np.nan][0]),
float([df['Координаты_t2'].split(' , ')[1] if df['Координаты_t2'] == df['Координаты_t2'] else np.nan][0])),
          axis=1)
result

,Дата операции_t1,Координаты_t1,Дата операции_t2,Координаты_t2,distance
0,2021-07-20 10:53:00,"55.7615174 , 37.6834498",2021-07-20 11:51:00,"55.7426108 , 37.5313959",9.753387
1,2021-07-20 10:53:00,"55.7615174 , 37.6834498",2021-07-20 06:38:00,"55.6734505 , 37.464511",16.856796
2,2021-07-20 10:53:00,"55.7615174 , 37.6834498",2021-07-19 09:51:00,"0 , 0",6640.944696
3,2021-07-20 10:53:00,"55.7615174 , 37.6834498",2021-07-19 09:51:00,"0 , 0",6640.944696
4,2021-07-20 10:53:00,"55.7615174 , 37.6834498",2021-07-19 09:50:00,"0 , 0",6640.944696
...,...,...,...,...,...
196685,2020-01-01 12:12:00,"55.8143736 , 37.3621306",2021-11-01 15:16:00,"55.673514 , 37.464937",16.947776
196686,2020-01-01 12:12:00,"55.8143736 , 37.3621306",2021-11-01 15:15:00,"55.673514 , 37.464937",16.947776
196687,2020-01-01 12:12:00,"55.8143736 , 37.3621306",2021-07-01 06:06:00,"0 , 0",6638.207120
196688,2020-01-01 12:12:00,"55.8143736 , 37.3621306",2021-04-01 13:54:00,"55.673594 , 37.465008",16.941223


### Задание 3b

В пандасе можно сохранять порядок строк в таблице, поэтому сразу отсортируем таблицу 123 по дате, чтобы сократить количество вычислений в функции.

In [7]:
table123 = table123.sort_values('Дата операции', ignore_index=1)

Для этих же целей разделим столбец Координаты на широту и долготу.

In [8]:
table123['Широта'] = table123['Координаты'].str.split(' , ').str[0].apply(float)
table123['Долгота'] = table123['Координаты'].str.split(' , ').str[1].apply(float)
table123.head(5)

,Дата операции,Координаты,Широта,Долгота
0,2020-01-01 12:12:00,"55.8143736 , 37.3621306",55.814374,37.362131
1,2020-01-01 12:13:00,"55.8143853 , 37.3621291",55.814385,37.362129
2,2020-01-01 20:53:00,"55.8149969 , 37.3614958",55.814997,37.361496
3,2020-01-05 06:10:00,"55.8112733333 , 37.70898",55.811273,37.708980
4,2020-01-05 11:19:00,"55.8112733333 , 37.70898",55.811273,37.708980


Создадим функцию для расчета параметров по заданным времени и координатам.

In [9]:
def get_row_params(ts, lat, long):
    
    table123_temp = table123.copy()
    table123_temp['Разность дат'] = ((ts - table123_temp['Дата операции']).dt.total_seconds() / 3600).round(1)
    table123_temp['Расстояние'] = table123_temp.apply(lambda df: distance(lat,long,df['Широта'],df['Долгота'])
                                                      ,axis=1).mul(1000).round(1)
    
    if ts <= table123_temp['Дата операции'][0]:
        ts_diff_prev = np.nan
        distance_prev = np.nan
        ts_diff_next = -table123_temp['Разность дат'][0]
        distance_next = table123_temp['Расстояние'][0]
        count_prev = 0
        count_next = table123_temp.query('`Разность дат` >= -24 * 365 and Расстояние <= 1000').shape[0]
        
    elif ts >= table123_temp['Дата операции'][table123.shape[0]-1]:
        ts_diff_prev = table123_temp['Разность дат'][table123.shape[0]-1]
        distance_prev = table123_temp['Расстояние'][table123.shape[0]-1]
        ts_diff_next = np.nan
        distance_next = np.nan
        count_prev = table123_temp.query('`Разность дат` <= 24 * 365 and Расстояние <= 1000').shape[0]
        count_next = 0
    
    else:
        ind_next_closest = table123_temp['Дата операции'].searchsorted(ts)
        
        ts_diff_prev = table123_temp.loc[ind_next_closest-1, 'Разность дат']
        distance_prev = table123_temp.loc[ind_next_closest-1, 'Расстояние']
        ts_diff_next = -table123_temp.loc[ind_next_closest, 'Разность дат']
        distance_next = table123_temp.loc[ind_next_closest, 'Расстояние']
        count_prev = table123_temp.loc[0:(ind_next_closest-1),:]\
        .query('`Разность дат` <= 24 * 365 and Расстояние <= 1000').shape[0]
        count_next = table123_temp.loc[ind_next_closest:(table123_temp.shape[0]-1),:]\
        .query('`Разность дат` >= -24 * 365 and Расстояние <= 1000').shape[0]
    
    params_list = [ts_diff_prev, distance_prev, ts_diff_next, distance_next, count_prev, count_next]
    return params_list

Запустим цикл.

In [10]:
#задаем параметры генерации данных
n = 30000
mean_ts = table123['Дата операции'].median()
days_std = table123['Дата операции'].std().days
mean_lat = table123['Широта'].median()
std_lat = 0.1
mean_long = table123['Долгота'].median()
std_long = 0.2

#создаем таблицу отчета
final_report = pd.DataFrame(columns=['Дата','Широта','Долгота','Время от прошлого события (ч)',
                                     'Расстояние от прошлого события (м)', 'Время до будущего события (ч)', 
                                     'Расстояние до будущего события (м)', 'Количество прошлых событий', 
                                     'Количество будущих событий', 'Доп. атрибут'])
#запускаем цикл
for i in tqdm(range(n)):
    data_gen = [mean_ts + dt.timedelta(round(np.random.normal(0,days_std),2)), #генерируем данные
                np.random.normal(mean_lat,std_lat),
                np.random.normal(mean_long,std_long)]
    params = get_row_params(data_gen[0],data_gen[1],data_gen[2]) #считаем параметры для этих данных

#рассчитываем доп.атрибут
    if params[0]/24/7 < 4 and params[2]/24/7 < 4 and \
       params[1] < 1000 and params[3] < 1000 and \
       params[4] > 0 and params[5] > 0:
        add_attr = [1]
    elif params[0] < 6 and params[2] < 6 and \
         params[1] > 100000 and params[3] > 100000 and \
         params[4] == 0 and params[5] == 0:
        add_attr = [0]
    else:
        add_attr = [random.randint(0,1)] # как по условию задачи - рандомно
#объединяем все поля в список и вставляем в датафрейм
    data_gen.extend(params)
    data_gen.extend(add_attr)
    final_report.loc[i] = data_gen
final_report.head(50)

100%|████████████████████████████████████████████████████████████████████████████| 30000/30000 [16:21<00:00, 30.57it/s]


,Дата,Широта,Долгота,Время от прошлого события (ч),Расстояние от прошлого события (м),Время до будущего события (ч),Расстояние до будущего события (м),Количество прошлых событий,Количество будущих событий,Доп. атрибут
0,2020-08-04 10:15:24,55.942961,37.891031,4.2,18537.6,41.4,18540.8,0,0,0
1,2020-08-10 12:53:48,55.951892,37.252338,7.0,32601.8,18.4,32470.6,0,0,0
2,2020-06-18 04:44:12,55.676505,37.484596,42.9,15687.3,1.0,15695.0,0,0,1
3,2021-11-27 21:32:12,55.751930,37.821598,509.1,6643235.4,82.0,8694.3,0,0,1
4,2021-02-28 12:39:24,55.865225,37.846559,21.5,10476.2,18.1,10470.5,0,0,0
5,2021-02-14 04:58:36,55.852384,37.721257,16.6,4619.6,1.9,4619.6,0,0,0
6,2022-06-10 12:10:36,55.777618,37.410830,4460.1,5641.0,NaN,NaN,0,0,1
7,2020-10-06 04:15:24,55.781324,38.189036,42.8,30221.5,1.9,31724.1,0,0,1
8,2020-07-11 18:10:36,55.750001,37.543474,12.1,12429.7,93.3,726057.4,0,3,0
9,2021-02-23 17:56:12,55.860229,37.446721,9.2,17253.8,16.9,18386.4,0,0,1
